## LangChain 출력 파서 (Output Parser)
- LangChain 출력 파서는 LLM이 생성한 텍스트를 원하는 형식(문자열, 리스트, JSON 등)으로 변환하거나 검증하는 도구이다.<br>
  자유 텍스트는 StrOutputParser, 구조화된 데이터는 PydanticOutputParser처럼 목적에 맞게 선택해 사용한다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] StrOutputParser
 : LLM 출력 텍스트를 그대로 반환하는 가장 단순한 파서

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1) 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 2) 프롬프트 템플릿 작성
prompt = ChatPromptTemplate.from_template("다음 주제로 간단한 시를 작성해줘 : {topic}.")

# 3) 출력 파서 (문자열 그대로)
parser = StrOutputParser()

# 4) 체인 구성: 프롬프트 → LLM → 문자열 파싱
chain = prompt | llm | parser

# 5) 실행
result = chain.invoke({"topic": "바다"})
print(result)

푸른 물결 속에 꿈을 담고,  
하늘과 맞닿은 수평선,  
바람에 실려 오는 파도 소리,  
속삭임처럼 마음을 적신다.

햇살 아래 반짝이는 모래,  
발자국 남기고 사라지는 시간,  
바다의 품에 안겨,  
영원히 잊지 못할 순간들.  

그리움의 노래가 흐르는 곳,  
파란 꿈을 꾸는 바다여,  
너와 나, 함께하는 이 순간,  
끝없이 넓은 사랑의 한 조각.  


In [3]:
type(result)  # str : StrOutputParser는 출력이 문자열(str)

str

In [4]:
# type(parser)
# dir(parser)  # 'invoke','parse'

In [5]:
ret = parser.invoke("하늘이 파란이유는?")
print(ret,type(ret))

하늘이 파란이유는? <class 'str'>


In [6]:
ret = parser.parse("하늘이 파란이유는?")
print(ret,type(ret))

하늘이 파란이유는? <class 'str'>


### [2] PydanticOutputParser

#### Pydantic이란?

: Pydantic은 Python에서 데이터 검증과 설정 관리를 쉽게 할 수 있도록 도와주는 데이터 검증 라이브러리이다. <br>
Python의 타입 힌트(type hint) 를 기반으로 동작한다. <br>
입력 데이터를 검사하고, 올바른 타입으로 자동 변환해 준다. <br>
잘못된 데이터가 들어오면 ValidationError 를 발생시켜 안전성을 높인다. <br>
한마디로 “데이터를 정해진 스키마(schema)대로 안전하게 다루게 해주는 도구”이다. <br>

### LangChain의 PydanticOutputParser는
: LLM이 출력한 텍스트를 Pydantic 모델로 변환해 주는 출력 파서이다. <br>

- 특징
LLM이 만든 JSON 같은 문자열을 자동으로 Python 객체로 바꿔준다. <br>
유효성 검사(validation) 기능이 내장되어 있어, 누락된 필드나 잘못된 데이터 타입을 잡아낼 수 있다. <br>
LLM이 JSON 형식을 틀리게 출력했을 때도, 자동으로 재시도하거나 오류를 알려줄 수 있다. 

In [7]:
from pydantic import BaseModel

# 데이터 스키마 정의
class User(BaseModel):
    id: int
    name: str
    age: int

# 올바른 데이터 → 정상 객체 생성
user = User(id=1, name="홍길동", age=25)
print(user)  
# User(id=1, name='홍길동', age=25)

# 잘못된 데이터 → 자동 변환 또는 에러
user2 = User(id="2", name="이몽룡", age="30")
print(user2)
# User(id=2, name='이몽룡', age=30)   ← 문자열이 int로 자동 변환됨

# 타입 불일치 → 에러 발생
# user3 = User(id="삼", name="성춘향", age="스무살")
# pydantic.error_wrappers.ValidationError 발생

id=1 name='홍길동' age=25
id=2 name='이몽룡' age=30


In [8]:
# 형변환
s = "2"
a = int(s)
print(a,type(a))

s = "2.0"
a = float(s)
print(a,type(a))


2 <class 'int'>
2.0 <class 'float'>


In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

# 1) 출력 데이터 구조 정의 (Pydantic 모델)
class Joke(BaseModel):
    setup: str = Field(description="농담의 전개 부분")
    punchline: str = Field(description="농담의 결론/웃긴 부분")

# 2) 파서 생성
parser = PydanticOutputParser(pydantic_object=Joke)

# 3) 프롬프트 템플릿 작성 (한글 버전)
prompt = ChatPromptTemplate.from_template(
    "주제 {topic}에 대한 농담을 하나 만들어줘.\n"
    "반드시 JSON 형식으로 대답해.\n"
    "{format_instructions}"
)

# 4) 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# 5) 체인 구성
chain = prompt | llm | parser

# 6) 실행
result = chain.invoke({"topic": "고양이", "format_instructions": parser.get_format_instructions()})

# 결과
print(result)
print("setup:", result.setup)
print("punchline:", result.punchline)

setup='왜 고양이는 컴퓨터를 잘 다룰까요?' punchline='왜냐하면 항상 마우스를 잡고 있으니까요!'
setup: 왜 고양이는 컴퓨터를 잘 다룰까요?
punchline: 왜냐하면 항상 마우스를 잡고 있으니까요!


In [10]:
type(result)  # Joke class의 객체

__main__.Joke

### [3] CSV Parser : CommaSeparatedListOutputParser
: LLM이 출력한 문자열을 쉼표(,) 기준으로 나눠 리스트로 변환해 주는 파서

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

prompt = PromptTemplate(
    template="5개만 알려줘. {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | output_parser

result = chain.invoke({"subject": "한국의 전통 민속 놀이는?"})
print(result)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['씨름', '윷놀이', '제기차기', '연날리기', '강강술래']


In [12]:
print(type(result))
print(result)

<class 'list'>
['씨름', '윷놀이', '제기차기', '연날리기', '강강술래']


In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
chain =  llm | parser
result = chain.invoke("한국의 전통 민속 놀이는?")
print(result)

한국의 전통 민속 놀이는 다양하고 각 지역마다 특색이 있습니다. 대표적인 민속 놀이로는 다음과 같은 것들이 있습니다:

1. **윷놀이**: 윷을 던져서 나온 결과에 따라 말을 이동시키는 게임으로, 가족이나 친구들과 함께 즐기는 대표적인 놀이입니다.

2. **제기차기**: 제기를 발로 차서 높이 띄우거나 멀리 보내는 놀이로, 주로 어린이들이 즐깁니다.

3. **투호**: 화살을 던져서 항아리나 통에 넣는 게임으로, 집중력과 정확성을 요구합니다.

4. **널뛰기**: 두 사람이 널판지를 양쪽에서 뛰어올리며 균형을 맞추는 놀이로, 주로 명절이나 특별한 날에 즐깁니다.

5. **고누**: 바둑판과 비슷한 판에서 돌을 놓고 상대의 돌을 잡는 전략 게임입니다.

6. **사방치기**: 네 개의 구역으로 나누어진 공간에서 공을 차거나 던져서 점수를 얻는 놀이입니다.

이 외에도 지역에 따라 다양한 민속 놀이가 존재하며, 이러한 놀이들은 한국의 전통 문화와 공동체 의식을 반영하고 있습니다.


### [4] JSON Parser : JsonOutputParser
: LLM이 생성한 결과를 JSON 형식으로 안전하게 파싱해주는 출력 파서.

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 1) 복잡한 JSON 스키마 정의
class CountryInfo(BaseModel):
    country: str = Field(description="국가 이름")
    capital: str = Field(description="수도 이름")
    population: int = Field(description="인구 수 (정수)")
    languages: List[str] = Field(description="사용되는 주요 언어 목록")
    major_cities: List[str] = Field(description="대표적인 도시들")

# 2) 출력 파서 준비
parser = JsonOutputParser(pydantic_object=CountryInfo)

# 3) 프롬프트 정의
prompt = PromptTemplate(
    template="""
다음 질문에 대해 반드시 JSON 형식으로 답변해 주세요.

질문: {question}

{format_instructions}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4) LLM 초기화
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 5) 실행
_input = prompt.format(question="대한민국의 수도, 인구, 언어, 주요 도시는 무엇인가요?")
output = model.invoke(_input)

# 6) 파싱
result = parser.parse(output.content)

print("LLM 출력:", output.content)
print("파싱 결과:", result)

LLM 출력: ```json
{
  "country": "대한민국",
  "capital": "서울",
  "population": 51780579,
  "languages": ["한국어"],
  "major_cities": ["부산", "인천", "대구", "대전", "광주"]
}
```
파싱 결과: {'country': '대한민국', 'capital': '서울', 'population': 51780579, 'languages': ['한국어'], 'major_cities': ['부산', '인천', '대구', '대전', '광주']}


In [15]:
_input

'\n다음 질문에 대해 반드시 JSON 형식으로 답변해 주세요.\n\n질문: 대한민국의 수도, 인구, 언어, 주요 도시는 무엇인가요?\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"country": {"description": "국가 이름", "title": "Country", "type": "string"}, "capital": {"description": "수도 이름", "title": "Capital", "type": "string"}, "population": {"description": "인구 수 (정수)", "title": "Population", "type": "integer"}, "languages": {"description": "사용되는 주요 언어 목록", "items": {"type": "string"}, "title": "Languages", "type": "array"}, "major_cities": {"description": "대표적인 도시들", "items": {"type": "string"}, "title": "Major Cities",

In [16]:
output

AIMessage(content='```json\n{\n  "country": "대한민국",\n  "capital": "서울",\n  "population": 51780579,\n  "languages": ["한국어"],\n  "major_cities": ["부산", "인천", "대구", "대전", "광주"]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 330, 'total_tokens': 394, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVALeJLxrX1GS2E1G352xSuh9SW6K', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--43b94e94-7c09-4acd-af06-ffb6231825ad-0', usage_metadata={'input_tokens': 330, 'output_tokens': 64, 'total_tokens': 394, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
chain = prompt | model | parser

result = chain.invoke({"question":"대한민국의 수도, 인구, 언어, 주요 도시는 무엇인가요?"})
print(result)  # 동일한 결과

{'country': '대한민국', 'capital': '서울', 'population': 51780579, 'languages': ['한국어'], 'major_cities': ['부산', '인천', '대구', '대전', '광주']}


### [5] StructuredOutputParser  
-> 랭체인 1.0버전에서 없어짐<br>
: LLM에 대한 답변을 dict/schema 형식으로 구성하고 key/value 쌍으로 갖는 여러 필드를 반환하고자 할 때 사용.<br>
StructuredOutputParser는 엄격한 검증은 필요 없고, 간단한 JSON 키-값만 빠르게 받고 싶을 때 가장 적합. <br>

- Pydantic/JSON 파서를 쓰면: <br>
올바른 JSON 스키마로 재시도 유도 <br>
타입 검증 실패 → 에러 감지 후 fallback 처리 <br>
결국 “불안정한 모델”도 안전한 구조화 데이터를 뽑아낼 수 있음

In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI

# 1) 출력 스키마 정의
response_schemas = [
    ResponseSchema(name="capital", description="국가의 수도"),
    ResponseSchema(name="population", description="국가의 인구 (정수)"),
    ResponseSchema(name="languages", description="국가에서 사용되는 주요 언어, 쉼표로 구분")
]

# 2) 파서 생성
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 3) 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""
다음 질문에 대해 반드시 JSON 형식으로 답변해 주세요.

질문: {question}

{format_instructions}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 4) LLM 초기화
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 5) 실행
_input = prompt.format(question="대한민국의 수도, 인구, 사용 언어를 알려주세요.")
output = model.invoke(_input)

# 6) 파싱
result = parser.parse(output.content)

print("LLM 출력:", output.content)
print("파싱 결과:", result) 

LLM 출력: ```json
{
	"capital": "서울",
	"population": "51780579",
	"languages": "한국어"
}
```
파싱 결과: {'capital': '서울', 'population': '51780579', 'languages': '한국어'}


In [20]:
_input

'\n다음 질문에 대해 반드시 JSON 형식으로 답변해 주세요.\n\n질문: 대한민국의 수도, 인구, 사용 언어를 알려주세요.\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"capital": string  // 국가의 수도\n\t"population": string  // 국가의 인구 (정수)\n\t"languages": string  // 국가에서 사용되는 주요 언어, 쉼표로 구분\n}\n```\n'

In [21]:
output

AIMessage(content='```json\n{\n\t"capital": "서울",\n\t"population": "51780579",\n\t"languages": "한국어"\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 122, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVANlNJJfNT3rlPb8OTgUKVsHccfa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f79ba965-7654-4a70-938f-383cfdf6a226-0', usage_metadata={'input_tokens': 122, 'output_tokens': 30, 'total_tokens': 152, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
chain = prompt | model | parser

result = chain.invoke({"question":"대한민국의 수도, 인구, 사용 언어를 알려주세요."})
print(result)  # 동일한 결과

{'capital': '서울', 'population': '51780579', 'languages': '한국어'}


### [6] PandasDataFrameOutputParser
: LLM의 출력 결과를 Pandas의 DataFrame으로 변환해주는 파서 <br>
LangChain 1.0 이후 버전에서 이 파서는 완전히 제거되었습니다. <br>
➡ 대신 StructuredOutputParser 또는 JsonOutputParser 를 사용해야 합니다. <br>

In [23]:
import json
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate(
    template="""
다음 질문에 대해 반드시 JSON 배열 형식으로만 답하세요.
코드 블록(```) 없이 순수 JSON만 출력하세요.

질문: {question}
""",
    input_variables=["question"],
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

_input = prompt.format(question="대한민국의 주요 도시와 인구를 알려주세요.")
output = model.invoke(_input)

# JSON → DataFrame
data = json.loads(output.content)
df = pd.DataFrame(data)

print("LLM 출력:", output.content)
print("\n변환된 Pandas DataFrame:")
df

LLM 출력: [
    {
        "도시": "서울",
        "인구": 9746000
    },
    {
        "도시": "부산",
        "인구": 3406000
    },
    {
        "도시": "인천",
        "인구": 2954000
    },
    {
        "도시": "대구",
        "인구": 2426000
    },
    {
        "도시": "대전",
        "인구": 1493000
    },
    {
        "도시": "광주",
        "인구": 1503000
    },
    {
        "도시": "울산",
        "인구": 1153000
    },
    {
        "도시": "세종",
        "인구": 350000
    }
]

변환된 Pandas DataFrame:


,도시,인구
0,서울,9746000
1,부산,3406000
2,인천,2954000
3,대구,2426000
4,대전,1493000
5,광주,1503000
6,울산,1153000
7,세종,350000


In [24]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("WHO_first9cols.csv")
df.head()

,Country,CountryID,Continent,Adolescent fertility rate (%),Adult literacy rate (%),Gross national income per capita (PPP international $),Net primary school enrolment ratio female (%),Net primary school enrolment ratio male (%),Population (in thousands) total
0,Afghanistan,1,1,151.0,28.0,NaN,NaN,NaN,26088.0
1,Albania,2,2,27.0,98.7,6000.0,93.0,94.0,3172.0
2,Algeria,3,3,6.0,69.9,5940.0,94.0,96.0,33351.0
3,Andorra,4,2,NaN,NaN,NaN,83.0,83.0,74.0
4,Angola,5,3,146.0,67.4,3890.0,49.0,51.0,16557.0


In [26]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain_classic.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 출력 포맷 지정
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)


# 파서 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항 출력
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [27]:
from langchain_classic.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import pandas as pd

# 모델 설정
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Parser 생성 
parser = PandasDataFrameOutputParser(dataframe=df)

# 프롬프트 설정
prompt = PromptTemplate(
    template="You are a data assistant. {format_instructions} Question: {query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성
chain = prompt | model | parser

# 실행
df_query = " Country column를 조회해 주세요."
result = chain.invoke({"query": df_query})

# 결과 출력
print("Result:\n", result)
print("Type:", type(result))
print('-'*50)

# 포맷 출력
format_parser_output(result)

Result:
 {'Country': 0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
197               Vietnam
198    West Bank and Gaza
199                 Yemen
200                Zambia
201              Zimbabwe
Name: Country, Length: 202, dtype: object}
Type: <class 'dict'>
--------------------------------------------------
{'Country': {0: 'Afghanistan',
             1: 'Albania',
             2: 'Algeria',
             3: 'Andorra',
             4: 'Angola',
             5: 'Antigua '
                'and '
                'Barbuda',
             6: 'Argentina',
             7: 'Armenia',
             8: 'Australia',
             9: 'Austria',
             10: 'Azerbaijan',
             11: 'Bahamas',
             12: 'Bahrain',
             13: 'Bangladesh',
             14: 'Barbados',
             15: 'Belarus',
             16: 'Belgium',
             17: 'Belize',
             18: 'Ben

### [7] DatetimeOutputParser
: LLM이 출력한 텍스트를 Python datetime 객체로 변환해 주는 역할을 한다. <br>
LangChain 1.0 이후 버전에서는 RegexParser 없어짐!!

In [31]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.output_parsers import DatetimeOutputParser
from langchain_openai import ChatOpenAI

# 출력 파서 
parser = DatetimeOutputParser()

# 프롬프트 정의
prompt = PromptTemplate(
    template="""
다음 질문에 대해 날짜/시간만 출력하세요.
⚠️ 반드시 {format_instructions} 형식으로 답변하세요.

질문: {question}
""",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

#  LLM 초기화
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 실행
_input = prompt.format(question="대한민국이 건국된 날짜는 언제인가요?")
output = model.invoke(_input)

#  파싱 : datetime 변환
dt = parser.parse(output.content)

print("LLM 출력:", output.content)
print("변환된 datetime 객체:", dt)
print("타입:", type(dt))

LLM 출력: 1948-08-15T00:00:00.000000Z
변환된 datetime 객체: 1948-08-15 00:00:00
타입: <class 'datetime.datetime'>


### [8] RegexParser
LangChain 1.0 이후 버전에서는 RegexParser 없어짐!!

In [32]:
# 파이썬 정규표현식
import re

text = "전화번호는 010-1234-5678 입니다."
result = re.findall(r"\d{3}-\d{4}-\d{4}", text)
print(result)  # ['010-1234-5678']

['010-1234-5678']


In [33]:
from langchain_classic.output_parsers import RegexParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 1) Regex 파서 정의
parser = RegexParser(
    regex=r"정답:\s*\(([a-d])\)",   # 괄호 안의 알파벳 추출
    output_keys=["answer"]
)

# 2) 프롬프트 정의 (get_format_instructions() 제거!)
prompt = PromptTemplate(
    template="""
다음 객관식 문제의 정답을 (a), (b), (c), (d) 중 하나로만 표시하세요.
출력 형식은 "정답: (a)" 와 같이 써주세요.

문제: {question}
""",
    input_variables=["question"],
)

# 3) 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 4) 실행
_input = prompt.format(question="대한민국의 수도는 어디인가요? (a) 도쿄 (b) 베이징 (c) 서울 (d) 평양")
output = model.invoke(_input)

# 5) 파싱
result = parser.parse(output.content)

print("LLM 출력:", output.content)
print("파싱 결과:", result)


LLM 출력: 정답: (c)
파싱 결과: {'answer': 'c'}


In [34]:
_input

'\n다음 객관식 문제의 정답을 (a), (b), (c), (d) 중 하나로만 표시하세요.\n출력 형식은 "정답: (a)" 와 같이 써주세요.\n\n문제: 대한민국의 수도는 어디인가요? (a) 도쿄 (b) 베이징 (c) 서울 (d) 평양\n'

In [35]:
output

AIMessage(content='정답: (c)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 88, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CVAPKbmhHJ4bSVMnFr0vtqAO17LnI', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5f53e928-c309-4153-96d1-3164197ff851-0', usage_metadata={'input_tokens': 88, 'output_tokens': 6, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [37]:
from langchain_classic.output_parsers import RegexParser

parser = RegexParser(
    regex=r"정답:\s*\(([a-d])\)",
    output_keys=["answer"]
)
result = parser.parse("정답: (c)")
print(result)  # {'answer': 'c'}

{'answer': 'c'}
